In [ ]:
import networkx as nx
% cd ..
from read_graph import read_graph
from common.pipeline import Pipeline
from common.feature_generators import *
from validation import *
from validation_datasets import *
from GSEA_validation import *

# Loading PPI graph
Graph, node_names = read_graph(directed=False)
print("Loaded graph:\n\t{} nodes\n\t{} edges".format(
    Graph.number_of_nodes(),
    Graph.number_of_edges()
    ))

#########################
# Computing node features
#########################

# The pipeline object takes as an argument the sequence of features we want
pipeline = Pipeline(Degree(), ExpectedDegree())
features = pipeline.apply(Graph, verbose=True)


#########################
# Learning
#########################

gene_query, gene_rank = get_query_and_rank(features, node_names, index=0)

#########################
# Validation
#########################

#Perform gene set enrichment analysis (GSEA) on a variety of gene sets directories
gene_sets_directories = [u'Cancer_Cell_Line_Encyclopedia', u'ChEA_2016', u'DrugMatrix', u'GeneSigDB', u'KEGG_2016', u'LINCS_L1000_Chem_Pert_down', u'LINCS_L1000_Chem_Pert_up', u'MSigDB_Computational', u'MSigDB_Oncogenic_Signatures', u'OMIM_Disease', u'OMIM_Expanded', u'PPI_Hub_Proteins', u'Panther_2016', u'Reactome_2016']
enrichr = enrichr_validation(gene_query, gene_rank=None, outdir="validation_results", gene_sets='KEGG_2016')
prerank = prerank_validation(gene_query, gene_rank, outdir="validation_results", gene_sets='KEGG_2016')

#Extract relevant gene lists

gene_ref = get_gene_ref(source="cancer")

compare_gene_lists(gene_query, gene_rank, gene_ref)



In [ ]:
gene_query, gene_rank = get_query_and_rank(features, node_names, index=0)


In [ ]:
len(gene_rank)

In [ ]:
N = len(gene_query_formatted)
n = 100
M = len(list(set(gene_query_formatted) & set(gene_ref_formatted)))
m = len(list(set(gene_query_formatted[0:n]) & set(gene_ref_formatted)))
print N,n,M,m

## Enrichr: http://amp.pharm.mssm.edu/Enrichr/#

In [ ]:
gene_list = ['CTLA2B', 'SCARA3', 'LOC100044683', 'CMBL', 'CLIC6', 'IL13RA1', 'TACSTD2', 'DKKL1', 'CSF1', 'CITED1']
gene_rank = 3*np.random.random(len(gene_list))
rnk = pd.DataFrame(np.array([gene_list, gene_rank]).T, columns = ['gene', 'score'])
rnk

#rnk = pd.DataFrame(np.array([gene_list]).T)
#rnk

In [ ]:
enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets='KEGG_2016', outdir='test', cutoff=0.05, format='png')
#enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets=u'ARCHS4_Cell-lines', outdir='test', cutoff=0.05, format='png')
enr.res2d[enr.res2d["Adjusted P-value"]<0.1]

## Prerank: http://software.broadinstitute.org/cancer/software/genepattern/modules/docs/GSEAPreranked/1

In [ ]:
pre_res = gp.prerank(rnk=rnk, gene_sets='KEGG_2016', outdir='prerank_report',format='png')
pre_res.res2d.head()

## Cancer gene Census (Controllability Paper)

In [ ]:
cancer_gene_census = pd.read_csv("cancer_gene_census.csv")
gene_symbols = cancer_gene_census.loc[:,"Gene Symbol"]
gene_names = cancer_gene_census.loc[:,"Name"]
gene_string = []
list_of_synonyms = cancer_gene_census.loc[:,"Synonyms"]
for synonyms in list_of_synonyms:
    if str(synonyms) != 'nan':        
        for synonym in synonyms.split(','):
            if synonym[:3] == "ENS":
                gene_string.append(synonym)
print "Number of genes = ", len(gene_symbols)

In [ ]:
#with open("mimTitles.txt", "r") as f:
gene_entrez = []
gene_symbols = []
gene_string = []
with open("validation_datasets/mim2gene.txt") as f:
    for line in f:
        if len(line.split('\t'))>1:
            print line
            if line.split('\t')[1] == "gene":
                if len(line.split('\t'))>2:
                    gene_entrez.append(line.split('\t')[2])
                    gene_symbols.append(line.split('\t')[3])
                    gene_string.append(line.split('\t')[4])

print "Nb of genes =", len(gene_string)

In [ ]:
#subset: "all" or "approved"
#molecule_type: "carrier", "enzyme", "target", "transporter"
def get_drug_bank(subset="all", molecule_type="target"):
    data = pandas.read_csv("validation_datasets/drugbank_%s_%s_polypeptide_ids.csv/all.csv"%(subset, molecule_type))
    data = data[data["Species"]=="Human"]
    gene_names = data["Gene Name"].values.tolist()
    protein_names = data["Name"].values.tolist()
    uniprot_ID = data["UniProt ID"].values.tolist()
    return gene_names, protein_names, uniprot_ID

gene_names, protein_names, uniprot_ID = get_drug_bank(subset="all", molecule_type="transporter")